#### Integrating the get_books() function into the Ebook_app Class
- below is the code for get_books() function 
- You will need to convert it into a method of the Ebook_app class:
1) add self as the first argument
2) once you have title, author, description and isbn, assign them to `self.title, self.author, self.description` and `self.isbn`.
3) remove any print statements
4) figure out how return should work. You have three cases: 
    - requests response was not 200, return False
    - no results for search term in google book store: `totalItems` is 0, return False
    - no results in the 10 results we requested,return False
    - A good result was found: return True


In [ ]:
# refactor this function into a method (I've already indented it)

    def get_books(search_term):

        url = "https://www.googleapis.com/books/v1/volumes"

        # create request url from parameters
        params = {"q": search_term, "maxResults": 10, 
                #"filter":"free-ebooks", 
                "orderBy":"relevance", 
                #"orderBy":"newest",
                "langRestrict":"en"}
        raw_response = requests.get(url, params=params) # make request (will be in json format)
        
        # bail out if we get an error
        if raw_response.status_code != 200:
            print(f"Error: {raw_response.status_code}")
            return None
        items = raw_response.json() # convert json to dictionary
        if items['totalItems'] == 0:
            return("Nothing found!")
        
        for book in items["items"]:
            volumeInfo = book["volumeInfo"] # dictionary of book info
            title = volumeInfo["title"]
            try:
                author = volumeInfo["authors"][0] # list of authors, only want first
            except KeyError: # not all books have an author
                author = "No author"
            try: # not all books have a description
                description = volumeInfo["description"]
            except KeyError:    
                description = "No description"

            try:
                identifier_list = volumeInfo["industryIdentifiers"]
            except KeyError:
                continue # skip this book if no chance for an ISBN
            else:
                isbn = None
                for identifier in identifier_list:
                    if identifier["type"] == "ISBN_13":
                        isbn = identifier["identifier"]
                    elif identifier["type"] == "ISBN_10":
                        isbn = identifier["identifier"]
                if isbn is None: 
                    continue

            # convert all words to latin to avoid unicode errors
            description = description.encode("latin-1", "ignore").decode("latin-1")
            title = title.encode("latin-1", "ignore").decode("latin-1")
            author = author.encode("latin-1", "ignore").decode("latin-1")
            
            # return a string with the book info for the first good book we find
            # this also breaks out of the loop
            return f"{title}\n{author}\nISBN:{isbn}\n{description}\n\n"   

### My solution for get_books() method

In [ ]:
# my solution of converting the function into a method and storing title, 
# author, isbn, description as class attributes and returning True/False as status

    def get_books(self, search_term):

        url = "https://www.googleapis.com/books/v1/volumes"

        # create request url from parameters
        params = {"q": search_term, "maxResults": 10, 
                "filter":"free-ebooks", 
                "orderBy":"relevance", 
                #"orderBy":"newest",
                "langRestrict":"en"}

        raw_response = requests.get(url, params=params) # make request (will be in json format)
        
        # bail out with False if we get an error
        if raw_response.status_code != 200:
            print(f"Error: {raw_response.status_code}")
            return False
        
        items = raw_response.json() # convert json to dictionary
        if items['totalItems'] == 0:
            return(False)
        
        for book in items["items"]:
            volumeInfo = book["volumeInfo"] # dictionary of book info
            title = volumeInfo["title"]
            try:
                author = volumeInfo["authors"][0] # list of authors, only want first
            except KeyError: # not all books have an author
                author = "No author"
            try: # not all books have a description
                description = volumeInfo["description"]
            except KeyError:    
                description = "No description"

            try:
                identifier_list = volumeInfo["industryIdentifiers"]
            except KeyError:
                continue # skip this book if no chance for an ISBN
            else:
                isbn = None
                for identifier in identifier_list:
                    if identifier["type"] == "ISBN_13":
                        isbn = identifier["identifier"]
                    elif identifier["type"] == "ISBN_10":
                        isbn = identifier["identifier"]
                if isbn is None: 
                    continue

                # convert all words to latin to avoid unicode errors
                self.description = description.encode("latin-1", "ignore").decode("latin-1")
                self.title = title.encode("latin-1", "ignore").decode("latin-1")
                self.author = author.encode("latin-1", "ignore").decode("latin-1")
                self.isbn = isbn
                return True # all went OK!

        return False # no good book found in requested number of results


### Open Ebook method
- this is a conversion of the function we used in step 6 into a method
- not that this assumes that we will have the isbn code stored in self.isbn

In [ ]:
    def open_ebook(self):
        # Create a Jinja2 environment with a template folder and load the template
        env = Environment(loader=FileSystemLoader('.')) # in current folder
        template = env.get_template('google_ebook_reader_template.html')

        # Create a context dictionary with the ISBN value
        context = {'isbn': self.isbn}

        # Render the template with the context
        rendered_template = template.render(context)

        # Create a temporary HTML file (you can use a unique name if needed)
        # this needs to be a absolute file path
        temp_html_file = os.path.abspath(f'{title}_by_{author}_ebook.html')

        # Write the rendered content to the temporary HTML file
        with open(temp_html_file, 'w') as file:
            file.write(rendered_template)

        # Open the temporary HTML file in a web browser
        # file:// is the equivalent to https:// for local html files
        webbrowser.open('file://' + temp_html_file) 

### Refactoring the search to use Google Books(2 pts)  
- copy your GUI class in the cell below where it says your code here
- you might want to split your view so you so you can read the refacturing steps while you edit the code

In [ ]:
# put your GUI class from Part B here and refactor it to use a google book based search
# instead of fuzzy search to generate a result. You will need these import 
import tkinter as tk
from tkinter.scrolledtext import ScrolledText
import requests
import webbrowser
from jinja2 import Environment, FileSystemLoader



# your code here


### Alternative: use my solution to the GUI class from Part B instead

In [ ]:
import tkinter as tk
from tkinter.scrolledtext import ScrolledText
import requests
import webbrowser
from jinja2 import Environment, FileSystemLoader

# Alternative: use my solution from Part B
class Ebook_app(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Ebook Search")

        # Create a Label and place it on the left (column 0)
        self.label = tk.Label(self, text="Search for Title:")
        self.label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        # Create an Entry Widget with a specific width (e.g., 30 characters)
        self.entry = tk.Entry(self, width=30)
        self.entry.grid(row=0, column=1, padx=10, pady=10, sticky="ew")

        # Create a Start Search Button and place it on the right (column 2)
        self.search_button = tk.Button(self, text="Start Search", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=10, pady=10, sticky="w")

        # for a row 2 add a label with Quality Threshold and a Scale widget from 0 to 100
        self.label = tk.Label(self, text="Quality Threshold:")
        self.label.grid(row=1, column=0, padx=10, pady=10, sticky="e")  
        self.scale = tk.Scale(self, from_=0, to=100, orient="horizontal") # start at 50
        self.scale.set(50)
        self.scale.grid(row=1, column=1, columnspan=2, padx=10, pady=10, sticky="ew")

        # Create a scrolled Text widget that spans all three columns and is 15 lines tall
        self.scrolled_text = ScrolledText(self, width=40, height=15)
        self.scrolled_text.grid(row=2, column=0, columnspan=3, padx=10, pady=10,)

        # Read books.csv file into a DataFrame (see Python refresher 4)
        self.df = pd.read_csv('books.csv')  

    def fuzzy_find(self, search_term, n=1):
        matches = process.extract(search_term, self.df["Title"], scorer=fuzz.token_set_ratio, limit=n)
        first_match = matches[0] # grab only first match, ignore n for now
        text = first_match[0] # the text of the first match
        match_score = first_match[1] # the similarity score of the first match
        index = first_match[2] # the index in the data frame of the first match
        #print(f"searched for {search_term}\nbest match: {text}, {index}, {match_score}") # DEBUG a list of tuples (text, similarity score, index)
        return text, index, match_score

    def start_search(self):
        search_term = self.entry.get()
        text, index, match = self.fuzzy_find(search_term)
        print(f"Best match for {search_term} is {text} with a score of {match}%")
        # if there is a > quality threshold match, print the book title, author, and summary,
        if match > self.scale.get():
            row = self.df.iloc[index]     # get the row of the best match using the index
            self.scrolled_text.insert(tk.END, f"The summary for {row['Title']} by {row['Author']} is:\n{row['Summary']}\n\n")
        else:
            self.scrolled_text.insert(tk.END, f"match of {match} is lower than {self.scale.get()}, no good match found!\n\n")
        
if __name__ == "__main__":
    app = Ebook_app()
    app.mainloop()

### Suggested steps for refactoring the GUI

- Restructure the GUI  (init)
    - keep the Label, Entry and Button widgets in row 0
    - remove the Label and Scale from row 1
    - set the ScrolledText widget to now be in row 1
    - add a button "Open Ebook" that with open_ebook as callback
    - put in in row 2 and make span all three columns
- remove the fuzzy_find() method
- copy/paste the open_ebook() method from above into the class
- copy/paste the get_books() method from earlier into the class
- completely rewrite the start_search() method
    - you can still use `search_term = self.entry.get()` to get the search term from the Entry widget
    - call self.get_books() with the search term and catch the True/False  status result
    - optionally: delete all text in the text widget
    - if you got True as status, insert the 3 results into the text widget, e.g.: `f"{self.title} by {self.author}:\n{self.description}\n\n"`
    - on False, insert a no match found message, e.g. `f"No match found for {search_term}"`

<p>

- I recommend that you now copy/paste all of this into the Final_app.py file and put a break point at the first line of start_search()
- check that your GUI looks OK
- enter a search term and hit Start Search
- Once it stops at the break point, step over each line and check that the variable values make sense
- Use step-into when it gets to get_books() and step through its code

<p>

- once everything works make a screenshot of 3 differnt searches and put them into this folder.




#### My solution for the final Google Books with GUI app (unhide this cell by clicking to the left of My)

In [ ]:
import tkinter as tk
from tkinter.scrolledtext import ScrolledText
import requests
import webbrowser
from jinja2 import Environment, FileSystemLoader

class Ebook_app(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Ebook Search")

        # Create a Label and place it on the left (column 0)
        self.label = tk.Label(self, text="Search for Title:")
        self.label.grid(row=0, column=0, padx=10, pady=10, sticky="e")

        # Create an Entry Widget with a specific width (e.g., 30 characters)
        self.entry = tk.Entry(self, width=30)
        self.entry.grid(row=0, column=1, padx=10, pady=10, sticky="ew")

        # Create a Start Search Button and place it on the right (column 2)
        self.search_button = tk.Button(self, text="Start Search", command=self.start_search)
        self.search_button.grid(row=0, column=2, padx=10, pady=10, sticky="w")

        # Create a scrolled Text widget that spans all three columns and is 15 lines tall
        self.scrolled_text = ScrolledText(self, width=40, height=15)
        self.scrolled_text.grid(row=1, column=0, columnspan=3, padx=10, pady=10,)

        # add a button to open the ebook in a browser
        self.open_button = tk.Button(self, text="Open Ebook", command=self.open_ebook)
        self.open_button.grid(row=2, column=0, columnspan=3, padx=10, pady=10, sticky="ew")
        
    def open_ebook(self):
        # Create a Jinja2 environment with a template folder and load the template
        env = Environment(loader=FileSystemLoader('.')) # in current folder
        template = env.get_template('google_ebook_reader_template.html')

        # Create a context dictionary with the ISBN value
        context = {'isbn': self.isbn}

        # Render the template with the context
        rendered_template = template.render(context)

        # Create a temporary HTML file (you can use a unique name if needed)
        # this needs to be a absolute file path
        temp_html_file = os.path.abspath(f'{title}_by_{author}_ebook.html')

        # Write the rendered content to the temporary HTML file
        with open(temp_html_file, 'w') as file:
            file.write(rendered_template)

        # Open the temporary HTML file in a web browser
        # file:// is the equivalent to https:// for local html files
        webbrowser.open('file://' + temp_html_file) 
    
    def get_books(self, search_term):
        url = "https://www.googleapis.com/books/v1/volumes"

        # create request url from parameters
        params = {"q": search_term, "maxResults": 10, 
                "filter":"free-ebooks", 
                "orderBy":"relevance", 
                #"orderBy":"newest",
                "langRestrict":"en"}

        raw_response = requests.get(url, params=params) # make request (will be in json format)
        
        # bail out with False if we get an error
        if raw_response.status_code != 200:
            print(f"Error: {raw_response.status_code}")
            return False
        
        items = raw_response.json() # convert json to dictionary
        if items['totalItems'] == 0:
            return(False)
        
        for book in items["items"]:
            volumeInfo = book["volumeInfo"] # dictionary of book info
            title = volumeInfo["title"]
            try:
                author = volumeInfo["authors"][0] # list of authors, only want first
            except KeyError: # not all books have an author
                author = "No author"
            try: # not all books have a description
                description = volumeInfo["description"]
            except KeyError:    
                description = "No description"

            try:
                identifier_list = volumeInfo["industryIdentifiers"]
            except KeyError:
                continue # skip this book if no chance for an ISBN
            else:
                isbn = None
                for identifier in identifier_list:
                    if identifier["type"] == "ISBN_13":
                        isbn = identifier["identifier"]
                    elif identifier["type"] == "ISBN_10":
                        isbn = identifier["identifier"]
                if isbn is None: 
                    continue

                # convert all words to latin to avoid unicode errors
                self.description = description.encode("latin-1", "ignore").decode("latin-1")
                self.title = title.encode("latin-1", "ignore").decode("latin-1")
                self.author = author.encode("latin-1", "ignore").decode("latin-1")
                self.isbn = isbn
                return True # all went OK!

        return False # no good book found in requested number of results

        url = "https://www.googleapis.com/books/v1/volumes"

        # create request url from parameters
        params = {"q": query, "maxResults": 40, 
                "filter":"free-ebooks", 
                "orderBy":"relevance", 
                #"orderBy":"newest",
                "langRestrict":"en"}

        raw_response = requests.get(url, params=params) # make request (will be in json format)
        
        # bail out with None if we get an error
        if raw_response.status_code != 200:
            print(f"Error: {raw_response.status_code}")
            return False
        
        items = raw_response.json() # convert json to dictionary

        if items['totalItems'] > 0:
            for book in items["items"]:
                volumeInfo = book["volumeInfo"] # dictionary of book info
                title = volumeInfo["title"]
                try:
                    author = volumeInfo["authors"][0] # list of authors, only want first
                except KeyError: # not all books have an author
                    continue
                try: # not all books have a description
                    description = volumeInfo["description"]
                except KeyError:    
                    description = "No description"

                try:
                    identifier_list = volumeInfo["industryIdentifiers"]
                except KeyError:
                    continue # skip this book if no chance for an ISBN
                else:
                    isbn = None
                    for identifier in identifier_list:
                        if identifier["type"] == "ISBN_13":
                            isbn = identifier["identifier"]
                        elif identifier["type"] == "ISBN_10":
                            isbn = identifier["identifier"]
                if isbn == None:        
                    continue

                # convert all words to latin to avoid unicode errors
                self.description = description.encode("latin-1", "ignore").decode("latin-1")
                self.title = title.encode("latin-1", "ignore").decode("latin-1")
                self.author = author.encode("latin-1", "ignore").decode("latin-1")
                self.isbn = isbn
                return True # all went OK!
            return False # no good book found in 40 results!
        else:
            return False # no books found

    def start_search(self):
        search_term = self.entry.get()
        if search_term != '':
            
            # Call the get_books method with the query and capture the status
            status = self.get_books(search_term)

            # delete Text widget contents
            self.scrolled_text.delete("1.0", "end") 
            
            # Depending on the return, use insert() to display the appropriate message
            if status == True: # got a book, display it
                self.scrolled_text.insert("end", f"{self.title} by {self.author}:\n{self.description}\n\n")
            
            else: # no book found or request error
                self.scrolled_text.insert("end", f"No match found for {search_term}")
                
if __name__ == "__main__":
    app = Ebook_app()
    app.mainloop()